This notebook is part of https://github.com/AudioSceneDescriptionFormat/splines, see also https://splines.readthedocs.io/.

[back to overview](hermite.ipynb)

# Non-Uniform Cubic Hermite Splines

We have already derived
[uniform cubic Hermite splines](hermite-uniform.ipynb),
where the parameter $t$ ranges from $0$ to $1$.

When we want use *non-uniform* cubic Hermite splines,
and therefore arbitrary ranges from $t_0$ to $t_1$,
we have (at least) two possibilities:

* Do the same derivations as in the *uniform* case,
  except when we previously evaluated an expression
  at the parameter value $t=0$,
  we now evaluate it at the value $t=t_0$.
  Of course we do the same with $t = 1 \to t = t_1$.

* Re-scale the *non-uniform* parameter using
  $t \to \frac{t - t_i}{t_{i+1} - t_i}$
  (which makes the new parameter go from $0$ to $1$)
  and then simply use the results from the *uniform* case.

The first approach leads to more complicated expressions
in the basis matrix and the basis polynomials,
but it has the advantage that the parameter value doesn't have to be re-scaled
each time when evaluating the spline for a given parameter
(which *might* be slightly more efficient).

The second approach has the problem that it doesn't actually work correctly,
but we will see that we can make a slight adjustment to fix that problem
(spoiler alert: we will have to multiply the tangent vectors by $\Delta_i$).

<div class="alert alert-info">

We show the second approach here,
but the first approach can be done very similarly,
with only very few changed steps.
The appropriate changes are mentioned below.

</div>

In [ ]:
import sympy as sp
sp.init_printing(order='rev-lex')

In [ ]:
from utility import NamedExpression, NamedMatrix

In [ ]:
t, t0, t1 = sp.symbols('t t:2')

In [ ]:
coefficients = sp.Matrix(sp.symbols('a:dbm0')[::-1])
b_monomial = sp.Matrix([t**3, t**2, t, 1]).T
b_monomial.dot(coefficients)

We use the humble cubic polynomial (with monomial basis)
to represent our curve segment $\boldsymbol{p}_0(t)$,
but we re-scale the parameter
to map $t_0 \to 0$ and $t_1 \to 1$:

In [ ]:
p0 = NamedExpression('pbm0', _.subs(t, (t - t0) / (t1 - t0)))

<div class="alert alert-info">

If you don't want to do the re-scaling,
simply un-comment the next line!

</div>

In [ ]:
#p0 = NamedExpression('pbm0', b_monomial.dot(coefficients))

Either way, this is our polynomial segment ...

In [ ]:
p0

... and it's derivative/velocity/tangent vectors:

In [ ]:
pd0 = p0.diff(t)
pd0

The next steps are very similar to what we did in the
[uniform case](hermite-uniform.ipynb),
except that we use $t_0$ and $t_1$ instead of
$0$ and $1$, respectively.

In [ ]:
x0 = p0.evaluated_at(t, t0).with_name('xbm0')
x1 = p0.evaluated_at(t, t1).with_name('xbm1')
xd0 = pd0.evaluated_at(t, t0).with_name('xdotbm0')
xd1 = pd0.evaluated_at(t, t1).factor().with_name('xdotbm1')

To simplify things,
we define a new symbol $\Delta_0 = t_1 - t_0$,
representing the duration of the current segment.
However, we only use this for simplifying the display,
further calculations are still carried out with $t_i$.

In [ ]:
delta = [
    (t1 - t0, sp.Symbol('Delta0')),
]

In [ ]:
display(x0, x1, xd0.subs(delta), xd1.subs(delta))

In [ ]:
M_H = NamedMatrix(r'{M_{\text{H},0}}', 4, 4)

In [ ]:
control_values_H = NamedMatrix(
    sp.Matrix([x0.name, x1.name, xd0.name, xd1.name]),
    M_H.name.I * coefficients)
control_values_H

In [ ]:
substitutions = x0, x1, xd0, xd1

In [ ]:
control_values_H.subs_symbols(*substitutions).subs(delta)

In [ ]:
M_H.I = sp.Matrix([
    [expr.expand().coeff(c) for c in coefficients]
    for expr in control_values_H.subs_symbols(*substitutions).name])
M_H.I.subs(delta)

In [ ]:
print(_.expr)

In [ ]:
M_H.factor().subs(delta)

In [ ]:
print(_.expr)

In [ ]:
b_H = NamedMatrix(r'{b_{\text{H},0}}', b_monomial * M_H.expr)
b_H.factor().subs(delta).simplify().T

Those are the *non-uniform* Hermite basis functions.
Not surprisingly, they are different for each segment,
because generally the values $\Delta_i$ are different
in the non-uniform case.

To quickly check whether the matrix $M_{H,0}$ does what we expect,
let's plot an example segment:

In [ ]:
import numpy as np

If we use the same API as for the other splines,
we can reuse the helper functions for plotting
from [helper.py](helper.py):

In [ ]:
from helper import plot_spline_2d, plot_tangents_2d

<div class="alert alert-info">

Note

The following code re-scales the parameter with:

```python
t = (t - begin) / (end - begin)
```

If you did *not* re-scale $t$ in the derivation above,
you'll have to remove this line.

</div>

In [ ]:
class HermiteSegment:
    
    def __init__(self, control_values, begin, end):
        array = sp.lambdify([t0, t1], M_H.expr)(begin, end)
        self.coeffs = array @ control_values
        self.grid = begin, end
    
    def evaluate(self, t):
        t = np.expand_dims(t, -1)
        begin, end = self.grid
        # If you derived M_H without re-scaling t, remove the following line:
        t = (t - begin) / (end - begin)
        return t**[3, 2, 1, 0] @ self.coeffs

In [ ]:
vertices = [0, 0], [5, 1]
tangents = [2, 3], [0, -2]

We can simulate the *uniform* case
by specifying a parameter range from $0$ to $1$:

In [ ]:
s1 = HermiteSegment([*vertices, *tangents], 0, 1)

In [ ]:
plot_spline_2d(s1, chords=False)
plot_tangents_2d(tangents, vertices)

But other ranges should work as well:

In [ ]:
s2 = HermiteSegment([*vertices, *tangents], 3, 5)

In [ ]:
plot_spline_2d(s2, chords=False)
plot_tangents_2d(tangents, vertices)

<div class="alert alert-info">

Note

If you did *not* re-scale $t$ in the beginning of the derivation,
you can use the matrix $M_{H,i}$ to calculate the monomial coefficients
of each segment (as shown in the example code above) and be done with it.

The following simplification does only apply
if you *did* re-scale $t$.

</div>

If you *did* re-scale $t$,
the basis matrix and the basis polynomials
will look very similar to the
[uniform case](hermite-uniform.ipynb),
but they are not quite the same.
This means that simply re-scaling the parameter
is not enough to correctly use the *uniform* results
for implementing *non-uniform* Hermite splines.

However, we can see that the only difference is
that the components associated with
$\dot{\boldsymbol{x}}_0$ and
$\dot{\boldsymbol{x}}_1$ are simply multiplied by $\Delta_0$.
That means if we re-scale the parameter *and*
multiply the given tangent vectors by $\Delta_0$,
we can indeed use the *uniform* workflow.

Just to make sure we are actually telling the truth, let's check that
the control values with scaled tangent vectors ...

In [ ]:
control_values_H_scaled = sp.Matrix([x0.name,
                                     x1.name,
                                     (t1 - t0) * xd0.name,
                                     (t1 - t0) * xd1.name])
control_values_H_scaled.subs(delta)

... really lead to the same result
as when using the *uniform* basis matrix:

In [ ]:
sp.simplify(sp.Eq(
    M_H.expr * control_values_H.name,
    sp.Matrix([[2, -2, 1, 1],
               [-3, 3, -2, -1],
               [0, 0, 1, 0],
               [1, 0, 0, 0]]) * control_values_H_scaled))

<div class="alert alert-info">

The following line will fail if you did *not* rescale $t$:

</div>

In [ ]:
assert _ == True

Long story short,
to implement a *non-uniform* cubic Hermite spline segment,
we can simply re-scale the parameter to a range from $0$ to $1$
(by substituting $t \to \frac{t - t_i}{t_{i+1} - t_i}$),
multiply both given tangent vectors by $\Delta_i = t_{i+1} - t_i$
and then simply use the implementation of
the *uniform* cubic Hermite spline segment.

Another way of looking at this is
to consider the *uniform* polynomial segment $\boldsymbol{u}_i(t)$
and its tangent vector (i.e. first derivative) $\boldsymbol{u}'_i(t)$.
If we want to know the tangent vector after substituting
$t \to \frac{t - t_i}{\Delta_i}$,
we have to use the [chain rule](https://en.wikipedia.org/wiki/Chain_rule)
(with the inner derivative being $\frac{1}{\Delta_i}$):

\begin{equation*}
\frac{d}{dt} \boldsymbol{u}_i\!\left(\frac{t-t_i}{\Delta_i}\right)
=
\frac{1}{\Delta_i} \boldsymbol{u}'_i\!\left(\frac{t-t_i}{\Delta_i}\right).
\end{equation*}

This means the tangent vectors have been shrunk by $\Delta_i$!
If we want to maintain the original lengths of our tangent vectors,
we can simply scale them by $\Delta_i$ beforehand.